In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import serial
import time
import threading
from pushbullet import Pushbullet

# Pushbullet 초기화 (자신의 Access Token 입력)
pb = Pushbullet("o.HZQ3TA7hMRp0ViXGPFMLyBBebxpehhZs")

# 메시지 전송 함수 (쓰레드로 비동기 처리)
def send_push_message(title, message):
    def push_task():
        pb.push_note(title, message)
        print("📨 메시지 전송 완료!")
    threading.Thread(target=push_task).start()

# 아두이노 연결
try:
    arduino = serial.Serial('COM5', 9600, timeout=1)
    time.sleep(2)
    print("🔌 아두이노 연결 완료")
except serial.SerialException as e:
    print("❌ 아두이노 연결 실패:", e)
    arduino = None

# YOLOv8 커스텀 모델 로드
det_model = YOLO("best.pt")
class_names = {0: "car"}  # 사용자 정의 클래스: car 1개

# 카메라 연결
cam = cv2.VideoCapture(0)
if not cam.isOpened():
    print("❌ 카메라 열기 실패")
    exit()
print("📷 카메라 연결 완료")

last_send_time = time.time()

while True:
    ret, frame = cam.read()
    if not ret:
        print("❌ 프레임 읽기 실패")
        break

    # 프레임 리사이즈 (해상도 조절)
    scale = 1280 / max(frame.shape)
    if scale < 1:
        frame = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)

    display_frame = frame.copy()

    # ROI(관심영역) 설정
    frame_height, frame_width = frame.shape[:2]
    roi_height = int(frame_height * 0.3)
    roi_y = (frame_height - roi_height) // 2
    roi = (0, roi_y, frame_width, roi_height)
    cv2.rectangle(display_frame, (roi[0], roi[1]), (roi[0]+roi[2], roi[1]+roi[3]), (255, 0, 0), 2)

    # YOLO 감지 수행
    results = det_model(frame, verbose=False)
    boxes = results[0].boxes
    car_detected = False

    for box in boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        label = class_names.get(cls_id, "unknown")
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        if label == "car" and conf > 0.7:
            if x1 < roi[0]+roi[2] and x2 > roi[0] and y1 < roi[1]+roi[3] and y2 > roi[1]:
                car_detected = True
                cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(display_frame, f"{label} {conf:.2f}", (x1, max(y1 - 5, 0)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                print("🚗 차량 감지됨")

    # 메시지/아두이노 전송 간격 제어
    current_time = time.time()
    if current_time - last_send_time >= 1.25 and arduino is not None:
        if car_detected:
            arduino.write(b'C')
            print("📤 아두이노 전송: 차량 감지")
            send_push_message("경고", "건널목에 차량이 있습니다.")
        else:
            arduino.write(b'N')
            print("📤 아두이노 전송: 차량 없음")
        last_send_time = current_time

    # 화면 출력
    cv2.imshow('cam', display_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 종료 처리
if arduino is not None:
    arduino.close()
cam.release()
cv2.destroyAllWindows()
